# **SIN 393 – Introduction to Computer Vision (2023-2)**

# Lecture 01 - Part 3 - Image classification - color features

Prof. João Fernando Mari ([*joaofmari.github.io*](https://joaofmari.github.io/))

---

## Importing the required libraries
---

In [1]:
import os

import numpy as np
from skimage import util, transform, filters, color, measure, morphology, io, exposure
from sklearn import model_selection, neighbors, metrics, preprocessing

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

### %matplotlib notebook
%matplotlib widget

## The dataset 
---
* Flowers detection dataset:

In [2]:
# 3 classes - 50 images in these classes
ds_path = 'data/flowers_toy'

# 3 classes - 6 images per class
#### ds_path = 'data/flowers_toy2'

### ***** Renomear os arquivos. Executar somente na primeira vez *****

In [3]:
# ***** Renomeia os arquivos. Executar somente uma vez *****

# # Iterate along the classes of the dataset
# for classe in classes_list:
#     # Listagem de todas as imagens na pasta daquela classe
#     filename_list = os.listdir(os.path.join(ds_path, classe))
#     filename_list.sort()
    
#     # Percorre os arquivos na pasta atual
#     for i, filename in enumerate(filename_list):
#         os.rename(os.path.join(ds_path, classe, filename), 
#                   os.path.join(ds_path, classe, f'{classe}{i:03}.{filename.split(".")[-1]}'))

## Loading the images from the dataset

In [4]:
# List of all folders in the folder 'ds_path' (classes)
classes_list = os.listdir(ds_path)

# List of all images in the dataset
image_list = []
# Lista of the image labels
label_list = []

# List of the image names
filename_list_ = []

# Iterate along the classes of the dataset
for classe in classes_list:
    
    # Listagem de todas as imagens na pasta daquela classe
    filename_list = os.listdir(os.path.join(ds_path, classe))

    filename_list.sort()
    
    # Percorre os arquivos na pasta atual
    for filename in filename_list:
        # Load the image
        img_temp = io.imread(os.path.join(ds_path, classe, filename))
        
        # Redimensiona a imagem para 64 x 64 
        img_temp = transform.resize(img_temp, (64, 64), anti_aliasing=True)
        
        # Append the image to a list
        image_list.append(img_temp)
        # Append the label to a list
        label_list.append(classe)
        # Append the file name to a list (para fins de visualização)
        filename_list_.append(filename)



In [ ]:
label_list = np.array(label_list)

# Lista com os rótulos das imagens        
print(label_list)

### Plotting the dataset images

* Plotting the N first images for each class.
* Important to start understanding the dataset.

In [ ]:
# Seleciona apenas as primeiras N imagens de cada classe
image_list_temp = []
filename_list_temp = []

# Itera pelo número de classes
for i in range(3):
    print(i, classes_list[i])
    # As 6 primeiras ocorrencias em que label é igual a 'i'.
    image_list_temp += [image_list[j] for j in np.where(label_list==classes_list[i])[0][:6]]
    filename_list_temp += [filename_list_[j] for j in np.where(label_list==classes_list[i])[0][:6]]

In [ ]:
fig, ax  = plt.subplots(3, 6, figsize=(9, 5))

for i, (image, filename) in enumerate(zip(image_list_temp, filename_list_temp)):
    ax[i//6, i%6].imshow(image, vmin=0, vmax=255)
    ax[i//6, i%6].set_title(str(filename), fontsize=10)
    ax[i//6, i%6].axis('off')

fig.tight_layout()
plt.show()

## Extracting some color based features from images
----

In [ ]:
# 2D array with the images features
# Each line stores information about an image. Each column stores one feature.
feature_mat = []

# Number of histogram bins 
n_bins = 32

for i, (image, label) in enumerate(zip(image_list, label_list)):
    # DEBUG
    print('Imagem {} - classe {}'.format(i, label))
    
    # Convert the image data type to float
    # img_float = util.img_as_float(image)
    img_float = image

    # Compute the histogram for each channel
    hist_r = exposure.histogram(img_float[:,:,0], nbins=n_bins)[0]
    hist_g = exposure.histogram(img_float[:,:,1], nbins=n_bins)[0]
    hist_b = exposure.histogram(img_float[:,:,2], nbins=n_bins)[0]

    # Concatenet the histogramns in one single feature vector
    feat_hist = np.concatenate((hist_r, hist_g, hist_b))
    ### print(feat_hist)

    # Append the feature vector to the feature matrix
    feature_mat.append(feat_hist)

In [ ]:
# Converte a lista de caracteristicas para um arranjo NumPy
feature_mat = np.array(feature_mat)

# Shape of the feature_map.
# Each row is a sample (image), and each column is a feature.
print(feature_mat.shape)

In [ ]:
# Algumas estatisticas sobre o conjunto de caracteristicas
with np.printoptions(precision=4, suppress=True):
    print('Histogram minimum values:')
    print(feature_mat.min(0))
    print('Histogram maximum values:')
    print(feature_mat.max(0))
    print('Histogram mean values:')
    print(feature_mat.mean(0))
    print('Histogram standard deviation values:')
    print(feature_mat.std(0))

### Plotando as caracteristicas computadas

In [10]:
# Seleciona apenas as primeiras N imagens de cada classe
feature_mat_temp = []
filename_list_temp = []

# Itera pelo número de classes
for i in range(3):
    # As 6 primeiras ocorrencias em que label é igual a 'i'.
    feature_mat_temp += [feature_mat[j] for j in np.where(label_list==classes_list[i])[0][:3]]
    filename_list_temp += [filename_list_[j] for j in np.where(label_list==classes_list[i])[0][:3]]

In [ ]:
fig, ax  = plt.subplots(3, 3, figsize=(9, 5))

for i, (feature, filename) in enumerate(zip(feature_mat_temp, filename_list_temp)):
    ## ax[i//6, i%6].imshow(image, vmin=0, vmax=255)
    ax[i//3, i%3].plot(feature)
    ax[i//3, i%3].set_title(str(filename), fontsize=10)
    ### ax[i//6, i%6].axis('off')

fig.tight_layout()
plt.show()

## Cross-validation - Hold-out
---


In [12]:
# Selecionamos apenas duas caracteristicas: Área e maior-eixo
feature_mat_ok = feature_mat

In [13]:
# Separa 20% do conjuto de completo para TESTES. 80% para treinamento 1.
X_train_1, X_test, y_train_1, y_test, file_train_1, file_test = model_selection.train_test_split(feature_mat_ok, 
                                                                                                 label_list, 
                                                                                                 filename_list_,
                                                                                                 test_size=0.2, 
                                                                                                 stratify=label_list,
                                                                                                 random_state=42)

In [14]:
# Separa 25% do conjuto de treinamento 1 para validação.
#   -> Equivale a 20% do conjunto completo. 0,2 / 0,8 = 0,25
X_train_2, X_val, y_train_2, y_val, file_train_2, file_val = model_selection.train_test_split(X_train_1, 
                                                                                              y_train_1, 
                                                                                              file_train_1,
                                                                                              test_size=0.25, 
                                                                                              stratify=y_train_1,
                                                                                              random_state=42)

In [ ]:
print(len(X_train_2))
print(len(X_val))
print(len(X_test))

## Feature normalization
---

In [ ]:
# Média das caracteristicas do conjunto de treinamento
X_train_2_mean = X_train_2.mean(0)

# Desvio padrão das caracteristicas do conjunto de treinamento
X_train_2_std = X_train_2.std(0)

with np.printoptions(precision=4, suppress=True):
    print(X_train_2.mean(0))
    print(X_train_2.std(0))

In [ ]:
# Transformada Normal de Caracteristicas (Manual)
# ----
# X_train_2_norm = (X_train_2 - X_train_2_mean) / X_train_2_std
# X_val_norm = (X_val - X_train_2_mean) / X_train_2_std
# X_test_norm = (X_test - X_train_2_mean) / X_train_2_std

# Transformada Normal de Caracteristicas (Sklearn)
# ----
scaler = preprocessing.StandardScaler().fit(X_train_2)
with np.printoptions(precision=4, suppress=True):
    print(f'Média:  \t {np.array(scaler.mean_)}')
    print(f'Desv. pad.: \t {np.array(scaler.scale_)}')

In [ ]:
X_train_2_norm = scaler.transform(X_train_2)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)

with np.printoptions(precision=4, suppress=True):
    print(f'Treino: \t {X_train_2_norm.mean():.4f} ± {X_train_2_norm.std():.4f}')
    print(f'Validação: \t {X_val_norm.mean():.4f} ± {X_val_norm.std():.4f}')
    print(f'Teste:   \t {X_test_norm.mean():.4f} ± {X_test_norm.std():.4f}')

## Optimizing hyperparameters in the validation set
---

In [19]:
k_list = [1, 3, 5, 7, 9]

In [ ]:
# Lista com as acurácias de traino
acc_train_list = []
# Lista com as acurácias de validação
acc_val_list = []

for k_ in k_list:
    # Constrói um classificador K-NN. K = k_
    clf = neighbors.KNeighborsClassifier(n_neighbors=k_, n_jobs=1)

    # Treinando o classificador
    clf.fit(X_train_2_norm, y_train_2)

    # Testando o classificador (usando o conjunto de validação)
    pred = clf.predict(X_val_norm)
    acc_val = metrics.accuracy_score(y_val, pred)
    
    acc_val_list.append(acc_val)
    
    # Testando o classificador (usando o conjunto de treino)
    # **** Apenas para comparar com o resultado da validação ****
    pred_train = clf.predict(X_train_2_norm)
    acc_train = metrics.accuracy_score(y_train_2, pred_train)
    
    acc_train_list.append(acc_train)  

In [ ]:
plt.figure(figsize=(9, 6))

plt.plot(k_list, acc_train_list, 'o', color='blue', label='treino')
plt.plot(k_list, acc_val_list, 'x', color='red', label='validação')
plt.xlabel("Valor de 'k'")
plt.ylabel("Acurácia")
plt.legend(loc='best')

plt.show()

In [ ]:
print('k \t acc. train \t acc. val')
print('----------------------------')
for k_, acc_t, acc_v in zip(k_list, acc_train_list, acc_val_list):
    print(f'{k_} \t {acc_t:.4f} \t {acc_v:.4f}')

k_best = k_list[np.argmax(acc_val_list)]
print(f'\nMelhor \'k\': {k_best} ({np.max(acc_val_list):.4f} acc.)')

## Evaluating the model over the test set
---

In [23]:
# Constrói um classificador K-NN. K = k_best
clf = neighbors.KNeighborsClassifier(n_neighbors=k_best)

# Treinando o classificador
clf.fit(X_train_2_norm, y_train_2)

# Testando o classificador (usando o conjunto de TESTES)
pred = clf.predict(X_test_norm)
acc_val = metrics.accuracy_score(y_test, pred)

### Confusion matrix and classification report

In [ ]:
print('\nConfusion matrix:')
print(metrics.confusion_matrix(y_test, pred))

print('\nClassification report:')
print(metrics.classification_report(y_test, pred))

### A detailed classification report

In [ ]:
for i, (y_test_, pred_, filename_) in enumerate(zip(y_test, pred, file_test)):
    print(f'{i} \t {filename_} \t {y_test_} \t {pred_} \t {(y_test_ == pred_)}')

## Bibliography
---
* GONZALEZ, R.C.; WOODS, R.E. **Digital Image Processing.** 3rd ed. Pearson, 2007.
* COSTA, L. DA F.; CESAR-JR., R. M. **Shape analysis and classification: theory and practice.** CRC Press, 2000. Chapter 8.
* Scikit-image documentation.
    * https://scikit-image.org/docs/stable/
* scikit-learn - User Guide.
    * https://scikit-learn.org/stable/user_guide.html